In [182]:
import pandas as pd
import pystac
import json
import re
from pathlib import Path
import subprocess


In [228]:
fe_text_file = Path(r"p:\11207608-coclico\docs\FE_text_titles.xlsx")
fe_text = pd.read_excel(fe_text_file, sheet_name="Sheet1")
fe_text

if fe_text.loc[fe_text["Collection"] == "ssl", "Styling"].isna:
    print('hell yeah')

# if fe_text.loc[fe_text["Collection"] == "ssl", "Styling"].empty:
#     print('hellyeah')
# else:
#     test = fe_text.loc[fe_text["Collection"] == "ssl", "Styling"].values[0]

# test_json = json.loads(test)

# test_json

hell yeah


In [156]:
# Set labels for FE
fe_labels = {
    "defense level": {
        "HIGH_DEFENDED_MAPS": "High Defended",
        "LOW_DEFENDED_MAPS": "Low Defended",
        "UNDEFENDED_MAPS": "No Defense"
    },
    "return period": {
        "static": "No Return Period",
        "1": "1 year",
        "5": "5 years",
        "10": "10 years",
        "20": "20 years",
        "50": "50 years",
        "100": "100 years",
        "200": "200 years",
        "500": "500 years",
        "1000": "1000 years"
    },
    "scenarios": {
        "None": "No Scenario",
        "SSP126": "SSP 126",
        "SSP245": "SSP 245",
        "SSP585": "SSP 585",
        "High_End": "High End",
        "Historical": "Historical",
        "RCP45": "RCP 4.5",
        "RCP85": "RCP 8.5"
    },
    "time": {
        "2010": "2010",
        "2030": "2030",
        "2050": "2050",
        "2100": "2100",
        "2150": "2150",
        # Added years with rounding to the nearest decade
        "2031": "2030",
        "2041": "2040",
        "2051": "2050",
        "2061": "2060",
        "2071": "2070",
        "2081": "2080",
        "2091": "2090",
        "2101": "2100",
        "2111": "2110",
        "2121": "2120",
        "2131": "2130",
        "2141": "2140",
        "2151": "2150"
    },
    "ensemble": {
        "msl_l": "16.7 Percentile",
        "msl_m": "50.0 Percentile",
        "msl_h": "83.3 Percentile"
    },
    "adaptation strategy": {
        "no_adaptation": "No Adaptation",
        "retreat": "Retreat",
        "protection": "Protect",
        "acc": "Accommodate",
        "protect_retreat": "Protect & Retreat"
    },
}

In [232]:
# Set up function for checking a collection based on a list of titles

def update_collection_FE_text(collection, fe_text_file, fe_labels):

    # Load FE_text excel file
    fe_text = pd.read_excel(fe_text_file, sheet_name="Sheet1")

    # Check if collection ID is in the FE_text
    collection_id = collection.id

    if not fe_text[fe_text["Collection"] == collection_id].empty:
        print('Collection found in FE-text, metadata will be updated...')

        # Change collection title
        collection.title = fe_text.loc[fe_text["Collection"] == collection_id, "Title"].values[0]

        # Change descriptions
        collection.description = fe_text.loc[fe_text["Collection"] == collection_id, "Markdown"].values[0]

        # Retrieve styling from FE_text
        # Check if excel cell contains data
        if fe_text.loc[fe_text["Collection"] == "ssl", "Styling"].isna: #fe_text.loc[fe_text["Collection"] == collection_id, "Styling"].empty:
            print(f"No styling found for collection {collection_id}, skipping...")
        else:        
            # Convert the JSON string into a Python dictionary
            styling = json.loads(fe_text.loc[fe_text["Collection"] == collection_id, "Styling"].values[0])

            # Iterate over the keys and values of the dictionary
            for key, value in styling.items():
                if key.startswith('deltares:'):
            
                    # Change the collection properties
                    collection.extra_fields[key] = value

        # Add front-end labels to the collection
        collection = add_FE_labels_to_collection(collection, fe_labels)

        # Save collection
        collection.save()

    else:
        print('Collection not found in FE-text, no metadata update needed...')

def add_FE_labels_to_collection(collection, fe_labels):
    # Initialize an empty dict to store the front-end labels
    summaries_labels = {}

    # Get summaries as a dictionary
    summaries_dict = collection.summaries.to_dict()

    # Iterate over the summaries dictionary
    for key, values in summaries_dict.items():
        if key in fe_labels:  # Check if the key has front-end labels
            label_map = fe_labels[key]  # Get the label mapping for the key
            # Map the original summary values to the front-end labels
            summaries_labels[key] = {value: label_map.get(value, value) for value in values}

    # Add the summaries_labels to the collection properties
    collection.extra_fields['summaries_labels'] = summaries_labels

    return collection

def update_catalog_FE_text(catalog, fe_text_file, fe_labels):
    
    # Load FE_text excel file
    fe_text = pd.read_excel(fe_text_file, sheet_name="Sheet1")

    # Get all collections from FE_text
    fe_collections = fe_text["Collection"].unique()

    # Loop through collections and update metadata
    for fe_collection_id in fe_collections:

        # Check if collection_id exists in the catalog
        if not catalog.get_child(fe_collection_id):
            print(f'Collection {fe_collection_id} not found in catalog, skipping...')
            continue

        print(f'Updating collection: {fe_collection_id}')
        collection = catalog.get_child(fe_collection_id)
        update_collection_FE_text(collection, fe_text_file, fe_labels)

        catalog.save()

def reset_catalog(folder="current", branch="main"):
    # Get the current working directory (should be in the 'notebooks' folder)
    current_dir = Path.cwd()

    # Go up two levels to reach the Git repo root
    repo_path = current_dir.parents[0]  # Adjust if necessary
    
    try:
        subprocess.run(
            ["git", "checkout", branch, "--", folder],
            check=True,
            cwd=repo_path  # Ensure git is executed from the repo root
        )
        print(f"✅ STAC Catalog and collections in folder: '{folder}' reset to match '{branch}' branch.")
    except subprocess.CalledProcessError as e:
        print(f"❌ Git command failed: {e}")


In [235]:
reset_catalog(folder="current")

✅ STAC Catalog and collections in folder: 'current' reset to match 'main' branch.


In [234]:
# hard-coded STAC templates
CUR_CWD = Path.cwd().parent
STAC_DIR = CUR_CWD / "current"  # .parent.parent

# Load catalog using pystac_client
catalog = pystac.Catalog.from_file(str(STAC_DIR / "catalog.json"))

update_catalog_FE_text(catalog,fe_text_file, fe_labels)

Updating collection: slp
Collection found in FE-text, metadata will be updated...
No styling found for collection slp, skipping...
Updating collection: ssl
Collection found in FE-text, metadata will be updated...
No styling found for collection ssl, skipping...
Updating collection: eesl
Collection found in FE-text, metadata will be updated...
No styling found for collection eesl, skipping...
Updating collection: cfhp_all_maps
Collection found in FE-text, metadata will be updated...
No styling found for collection cfhp_all_maps, skipping...
Updating collection: cet
Collection found in FE-text, metadata will be updated...
No styling found for collection cet, skipping...
Updating collection: cba
Collection found in FE-text, metadata will be updated...
No styling found for collection cba, skipping...
Updating collection: be_maps
Collection found in FE-text, metadata will be updated...
No styling found for collection be_maps, skipping...
Updating collection: pp_maps
Collection found in FE-t

In [221]:
collection

<Collection id=slp>

In [202]:
print(collection.summaries)

In [136]:
collection

<Collection id=slp>

In [119]:
import json
# The JSON string you provided
json_string = '''{
    "deltares:units": "m",
    "deltares:plotSeries": "scenario",
    "deltares:plotxAxis": "time",
    "deltares:plotType": "stacked_series",
    "deltares:min": -0.2,
    "deltares:max": 0.5,
    "deltares:linearGradient": [
        {
            "color": "#ffdbb3",
            "offset": "0.000%",
            "opacity": 5
        },
        {
            "color": "#ffcf99",
            "offset": "28.571%",
            "opacity": 50
        },
        {
            "color": "#ffb666",
            "offset": "50.000%",
            "opacity": 50
        },
        {
            "color": "#ff8600",
            "offset": "71.429%",
            "opacity": 50
        },
        {
            "color": "#cc6b00",
            "offset": "100.000%",
            "opacity": 50
        }
    ]
}'''

# Convert the JSON string into a Python dictionary
data = json.loads(json_string)

In [121]:
# Iterate over the keys and values of the dictionary
for key, value in data.items():
    if key.startswith('deltares:'):
        
        # Change the collection properties
        collection.extra_fields[key] = value

collection

<Collection id=slp>

In [129]:
collection.summaries_labels = 'test'

In [214]:
# Extrac collection summaries
collection.summaries.to_dict()

{'scenarios': ['high_end', 'ssp126', 'ssp245', 'ssp585'],
 'ensemble': ['msl_h', 'msl_m', 'msl_l'],
 'time': ['2031',
  '2041',
  '2051',
  '2061',
  '2071',
  '2081',
  '2091',
  '2101',
  '2111',
  '2121',
  '2131',
  '2141',
  '2151']}